# Proyecto DL

### Librerias

In [ ]:
!pip install flair
!pip install rouge
!pip install sacrebleu

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
from rouge import Rouge
import sacrebleu
from flair.models import SequenceTagger
from flair.data import Sentence

### Modelo y función de translate

https://huggingface.co/Helsinki-NLP/opus-mt-es-en

In [ ]:
model_name = 'Helsinki-NLP/opus-mt-es-en'

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
#model

In [ ]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    translated = model.generate(**inputs)

    translation = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translation

In [ ]:
input = "He aprendido mucho sobre modelos de lenguaje en este proyecto"
reference = "I have learned a lot about language models in this project"
candidate = translate(input)

print("Candidate:", candidate)
print("Referencia:", reference)

### Metrica Rouge

https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499

https://github.com/pltrdy/rouge

R: reference

C: candidate


In [ ]:
rouge = Rouge()
scores = rouge.get_scores(candidates, english_sample)
scores

### Metrica bleu

https://aclanthology.org/P02-1040.pdf

In [ ]:
bleu_score = sacrebleu.corpus_bleu([candidate], [[reference]])
bleu_score.score

### Dataset

https://www.dropbox.com/scl/fi/xtvls58le65vxjfkmuv9x/Sentence-pairs-in-Spanish-English-2024-07-03.tsv?rlkey=3la4vmua8d5f5ltdknfhmcmyt&st=6cdj4k63&dl=0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Nose que significa ese number, en vola id de la traduccion tb?

In [ ]:
column_names = ['id_es', 'spanish', 'id_en', 'english']

data = pd.read_csv('/content/drive/MyDrive/Sentence pairs in Spanish-English - 2024-07-03.tsv', sep='\t', names=column_names, header=None)

### NER(Named-entity recognition)

In [ ]:
#data.head()
data.describe()

In [ ]:
import sacrebleu


# Extracción de las columnas necesarias
spanish_inputs = data['spanish'].tolist()[0:5]
candidates = []
for input in spanish_inputs:
    candidates.append(translate(input))

references = data['english'].tolist()[0:5]

# Bleu
def calculate_bleu(candidate_list, reference_list):
    bleu_score = sacrebleu.corpus_bleu(candidate_list, [reference_list])
    print(f"BLEU Score: {bleu_score.score}")
    return bleu_score.score


# Rogue
def calculate_rogue(candidates, references):
    scores = rouge.get_scores(candidates, references)
    fscores = [score["rouge-l"]["f"] for score in scores]
    avg = sum(fscores) / len(fscores)
    print(f"Rogue Scores: {avg}")


print(references)
print(candidates)
calculate_bleu(candidates, references)
calculate_rogue(candidates, references)

In [ ]:
# tagger = SequenceTagger.load("flair/upos-english-fast")

def text_to_pos(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    pos_list = [word.value for word in sentence.labels]
    return " ".join(pos_list)

def list_to_pos(list):
    pos_list = []
    for text in list:
        pos_list.append(text_to_pos(text))
    return pos_list

def compare_pos(candidates, labels):
    for i in range(len(candidates)):
        candidate = candidates[i]
        label = labels[i]
        candidate_pos = text_to_pos(candidate)
        label_pos = text_to_pos(label)
        if (candidate != label):
          print(f"Candidate: {candidate}")
          print(f"POS candidate: {candidate_pos}")
          print(f"Reference: {label}")
          print(f"POS candidate: {label_pos}")
          print(f"Match? {candidate_pos == label_pos}\n")


candidates_pos = list_to_pos(candidates)
references_pos = list_to_pos(references)
print(candidates)
print(references)
calculate_bleu(candidates, references)
calculate_rogue(candidates, references)
print(candidates_pos)
print(references_pos)
calculate_bleu(candidates_pos, references_pos)
calculate_rogue(candidates_pos, references_pos)

https://github.com/marcelomendoza/IIC3670/blob/main/codes/8%20-%20SEQUENCE%20LABELING%2C%20POS%20TAGGING%20Y%20NER.ipynb